In [1]:
import sys
sys.path.append('../')
from Roma import graphbuilder_v2
from availability_estimation import *


/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LO

In [2]:
lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
city = lo_polygon.union(spb_polygon).to_crs(epsg=4326)

russia = ox.geocode_to_gdf("Russia")

In [3]:

citygraph = graphbuilder_v2.get_graph_from_polygon(city, crs=32636,country_polygon=russia)


/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 52 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
/Users/polina/Desktop/github/transport_frames/Polina/../Roma/graphbuilder_v2.py:185: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  edges = edges.explode()


In [4]:
import geopandas as gpd

regions = gpd.read_file('russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

In [5]:
carcas = graphbuilder_v2.get_carcas(citygraph,regions,city)
n,e = momepy.nx_to_gdf(carcas)
exits = n[n['exit']==1]

/var/folders/ch/1hzm9xv12jg458kt9c3nl6_h0000gn/T/ipykernel_48787/3858161607.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  carcas = graphbuilder_v2.get_carcas(citygraph,regions,city)


In [6]:
mapping = {node : data['nodeID'] for node,data in carcas.nodes(data=True)} # inside get_carcas
carcas = nx.relabel_nodes(carcas, mapping)

In [ ]:
exits['buf'] = exits.geometry.buffer(15000)
exits.set_geometry('buf').explore()

In [74]:
external_node_matrix = [
    [0.12, 0.12, 0.12, 0.12, 0.12, 0.12],  # 2.1.1
    [0.10, 0.10, 0.10, 0.10, 0.10, 0.10],  # 2.1.2
    [0.08, 0.08, 0.08, 0.08, 0.08, 0.08],  # 2.1.3
    [0.07, 0.07, 0.07, 0.07, 0.07, 0.07],  # 2.2.1
    [0.06, 0.06, 0.06, 0.06, 0.06, 0.06],  # 2.2.2
    [0.05, 0.05, 0.05, 0.05, 0.05, 0.05]   # 2.2.3
]

In [112]:
def get_weight(start,end,exit):
    dict = {1.1: 0, 1.2: 1, 1.3: 2, 2.1: 3, 2.2: 4, 2.3: 5}
    if exit == 1:
        matrix = [
                [0.12, 0.12, 0.12, 0.12, 0.12, 0.12],  # 2.1.1
                [0.10, 0.10, 0.10, 0.10, 0.10, 0.10],  # 2.1.2
                [0.08, 0.08, 0.08, 0.08, 0.08, 0.08],  # 2.1.3
                [0.07, 0.07, 0.07, 0.07, 0.07, 0.07],  # 2.2.1
                [0.06, 0.06, 0.06, 0.06, 0.06, 0.06],  # 2.2.2
                [0.05, 0.05, 0.05, 0.05, 0.05, 0.05]   # 2.2.3
            ]
    else:
        
        matrix = [
                [0.08, 0.08, 0.08, 0.08, 0.08, 0.08],  # 2.1.1
                [0.07, 0.07, 0.07, 0.07, 0.07, 0.07],  # 2.1.2
                [0.06, 0.06, 0.06, 0.06, 0.06, 0.06],  # 2.1.3
                [0.05, 0.05, 0.05, 0.05, 0.05, 0.05],  # 2.2.1
                [0.04, 0.04, 0.04, 0.04, 0.04, 0.04],  # 2.2.2
                [0.02, 0.02, 0.02, 0.02, 0.02, 0.02]   # 2.2.3
            ]
    return matrix[dict[end]][dict[start]]

In [121]:
print(e.iloc[391]['geometry'])

LINESTRING (306612.378989 6490384.596683, 306669.983529 6490417.180428, 306963.931125 6490587.647634, 307168.643524 6490777.53682, 307730.582516 6491509.703186, 307834.177345 6491636.990535)


In [114]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
import networkx as nx
import matplotlib.pyplot as plt
import contextily as ctx

data = []
flag=1
for i1, start_node in exits.iterrows():
    if flag != 0:
        for i2, end_node in exits.iterrows():

            if i1 == i2:
                continue
            if pd.notna(start_node['border_region']) and start_node['border_region'] == end_node['border_region']:
                continue
            if start_node.geometry.buffer(15000).intersects(end_node.geometry.buffer(15000)) and (pd.isna(start_node['exit_country']) == pd.isna(end_node['exit_country'])):
                continue
            if start_node['exit_country'] == 1 and end_node['exit_country'] == 1:
                continue

            weight = get_weight(start_node['ref_type'],end_node['ref_type'],end_node['exit_country'])

            try:
                # Find the shortest path using Dijkstra's algorithm
                path = nx.dijkstra_path(carcas, i1, i2, weight='time_min')
            except nx.NetworkXNoPath:
                continue

            # Extract the path coordinates and concatenate geometries
            for j in range(len(path) - 1):
                edge_data = carcas.get_edge_data(path[j], path[j + 1])
                
                for key,value in edge_data.items():
                    
                    g = value['geometry']
                    data.append({
                    'start_node': i1,
                    'end_node': i2,
                    'start_ref': start_node['ref_type'],
                    'end_ref': end_node['ref_type'],
                    'geometry': g,
                    'start_region': start_node['border_region'],
                    'end_region': end_node['border_region'],
                    'end_exit': end_node['exit_country'],
                    'weight':weight
                })

            
            # merged_linestring = shapely.LineString([point for linestring in path_coords for point in linestring.coords])
            # line = shapely.geometry.LineString(merged_linestring)
            

            
# Create the GeoDataFrame with the collected data
roads_gdf = gpd.GeoDataFrame(data, crs=exits.crs)
roads_gdf


,start_node,end_node,start_ref,end_ref,geometry,start_region,end_region,end_exit,weight
0,77,4725,2.3,2.2,"LINESTRING (288110.106 6502090.938, 288138.916...",relation/155262,relation/89331,NaN,0.04
1,77,4725,2.3,2.2,"LINESTRING (288901.037 6501625.268, 289010.782...",relation/155262,relation/89331,NaN,0.04
2,77,4725,2.3,2.2,"LINESTRING (298655.383 6501306.953, 298700.106...",relation/155262,relation/89331,NaN,0.04
3,77,4725,2.3,2.2,"LINESTRING (298922.635 6501143.449, 298935.416...",relation/155262,relation/89331,NaN,0.04
4,77,4725,2.3,2.2,"LINESTRING (301988.839 6498389.991, 301994.541...",relation/155262,relation/89331,NaN,0.04
...,...,...,...,...,...,...,...,...,...
239664,21825,21721,1.3,2.3,"LINESTRING (217244.917 6554876.873, 217134.142...",NaN,relation/155262,NaN,0.02
239665,21825,21721,1.3,2.3,"LINESTRING (216158.873 6555158.544, 216085.012...",NaN,relation/155262,NaN,0.02
239666,21825,21721,1.3,2.3,"LINESTRING (215605.788 6555503.324, 215578.839...",NaN,relation/155262,NaN,0.02
239667,21825,21721,1.3,2.3,"LINESTRING (215386.082 6555389.423, 215237.345...",NaN,relation/155262,NaN,0.02


In [145]:
print(roads_gdf.iloc[2]['geometry'])

LINESTRING (298655.383046 6501306.95326, 298700.105565 6501301.808081, 298734.730276 6501295.132775, 298779.417535 6501283.330951, 298790.043301 6501279.078585, 298799.468862 6501274.018183, 298811.62661 6501264.711714, 298822.293075 6501255.047286, 298843.99856 6501233.12254, 298864.380682 6501210.931634, 298917.745729 6501149.111735, 298922.634683 6501143.449284)


In [130]:
a = roads_gdf.groupby('geometry').agg({'weight': 'sum'}).reset_index().set_geometry('geometry').to_crs(4326)

In [168]:
import folium
m = folium.Map(zoom_start=10)

ma =  a['weight'].max()
def style_function(feature):
    weight = feature['properties']['weight']/ma
    return {
        'color': 'green',  # You can set any color you want
        'weight': weight * 20 # Adjust the multiplier as needed
    }

# Add the GeoDataFrame to the map, customizing the style
folium.GeoJson(a, style_function=style_function).add_to(m)


# Add the GeoDataFrame to the map
# folium.GeoJson(a).add_to(m)

folium.GeoJson(e[e['reg']==1],style_function=lambda feature:{'color':'red'}).add_to(m)
folium.GeoJson(e[e['reg']==2],style_function=lambda feature:{'color':'blue','weight':1}).add_to(m)
for _, row in n[n['exit'] == 1].to_crs(4236).iterrows():
    folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=5, color='blue', fill=True, fill_color='blue').add_to(m)
m

In [108]:
from tqdm import tqdm

In [ ]:

# Visualization with Matplotlib and Contextily
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the roads
roads_gdf.plot(ax=ax, edgecolor='red', linewidth=2)

# Add the basemap
ctx.add_basemap(ax, crs=roads_gdf.crs.to_string(), source=ctx.providers.Stamen.TonerLite)

# Add labels and title
plt.title("Paths between Nodes with Real Geometries")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()